**References:**  

https://www.datacamp.com/community/tutorials/finance-python-trading

https://github.com/datacamp/datacamp-community-tutorials/blob/master/Python%20Finance%20Tutorial%20For%20Beginners/Python%20For%20Finance%20Beginners%20Tutorial.ipynb

https://pypi.python.org/pypi/fix-yahoo-finance

http://www.learndatasci.com/python-finance-part-yahoo-finance-api-pandas-matplotlib/

### Python for Finance: Stock Portfolio Analyses

https://towardsdatascience.com/python-for-finance-stock-portfolio-analyses-6da4c3e61054

In [1]:
# Import initial libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__)  # requires version >= 1.9.0

init_notebook_mode(connected=True)

5.22.0


In [3]:
# Import the Sample worksheet with acquisition dates and initial cost basis:

portfolio_df = pd.read_excel("Sample stocks acquisition dates_costs.xlsx", sheet_name="Sample")

portfolio_df.head(10)

,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year
0,2013-02-07,AAPL,125,65.40,8175.00,2017-12-29
1,2014-02-27,JNJ,100,81.90,8190.00,2017-12-29
2,2013-06-04,MCD,100,84.99,8499.00,2017-12-29
3,2015-12-14,MTCH,600,13.63,8178.00,2017-12-29
4,2016-01-14,NFLX,75,108.71,8153.25,2017-12-29
5,2013-08-14,WMT,125,68.30,8537.50,2017-12-29
6,2013-12-13,FB,150,53.32,7998.00,2017-12-29
7,2015-01-05,TWTR,225,36.38,8185.50,2017-12-29


In [4]:
# Confirm that you have 8 values for each column.

portfolio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Acquisition Date  8 non-null      datetime64[ns]
 1   Ticker            8 non-null      object        
 2   Quantity          8 non-null      int64         
 3   Unit Cost         8 non-null      float64       
 4   Cost Basis        8 non-null      float64       
 5   Start of Year     8 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 512.0+ bytes


In [5]:
# Date Ranges for SP 500 and for all tickers
# Modify these date ranges each week.

# The below will pull back stock prices from 2010 until end date specified.
start_sp = datetime.datetime(2013, 1, 1)
end_sp = datetime.datetime(2018, 3, 9)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2017, 12, 29)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2013, 1, 1)
stocks_end = datetime.datetime(2018, 3, 9)

In [6]:
# Leveraged from the helpful Datacamp Python Finance trading blog post.

from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()  # <== that's all it takes :-)

sp500 = pdr.get_data_yahoo("^GSPC", start_sp, end_sp)

sp500.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,1462.420044,4202600000
2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,1459.369995,3829730000
2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,1466.469971,3424290000
2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,1461.890015,3304970000
2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,1457.150024,3601600000


In [7]:
sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-02,2658.889893,2696.250000,2647.320068,2691.250000,2691.250000,3891250000
2018-03-05,2681.060059,2728.090088,2675.750000,2720.939941,2720.939941,3733640000
2018-03-06,2730.179932,2732.080078,2711.260010,2728.120117,2728.120117,3389030000
2018-03-07,2710.179932,2730.600098,2701.739990,2726.800049,2726.800049,3406690000
2018-03-08,2732.750000,2740.449951,2722.649902,2738.969971,2738.969971,3206040000


In [10]:
# Create a dataframe with only the Adj Close column as that's all we need for this analysis.

sp_500_adj_close = sp500[["Adj Close"]].reset_index()

In [11]:
sp_500_adj_close.tail()

,Date,Adj Close
1300,2018-03-02,2691.250000
1301,2018-03-05,2720.939941
1302,2018-03-06,2728.120117
1303,2018-03-07,2726.800049
1304,2018-03-08,2738.969971


In [12]:
# Adj Close for the EOY in 2017 in order to run comparisons versus stocks YTD performances.

sp_500_adj_close_start = sp_500_adj_close[sp_500_adj_close["Date"] == end_of_last_year]
sp_500_adj_close_start

,Date,Adj Close
1258,2017-12-29,2673.610107


In [13]:
# Generate a dynamic list of tickers to pull from Yahoo Finance API based on the imported file with tickers.
tickers = portfolio_df["Ticker"].unique()
tickers

array(['AAPL', 'JNJ', 'MCD', 'MTCH', 'NFLX', 'WMT', 'FB', 'TWTR'],
      dtype=object)

In [14]:
# Stock comparison code


def get(tickers, startdate, enddate):
    def data(ticker):
        return pdr.get_data_yahoo(ticker, start=startdate, end=enddate)

    datas = map(data, tickers)
    return pd.concat(datas, keys=tickers, names=["Ticker", "Date"])


all_data = get(tickers, stocks_start, stocks_end)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FB']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TWTR']: Exception('%ticker%: No timezone found, symbol may be delisted')
/tmp/ipykernel_1031/2562536656.py:9: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the e

In [15]:
all_data.head()

Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
AAPL   2013-01-02  19.779285  19.821428  19.343929  19.608213  16.725033   
       2013-01-03  19.567142  19.631071  19.321428  19.360714  16.513929   
       2013-01-04  19.177500  19.236786  18.779642  18.821428  16.053942   
       2013-01-07  18.642857  18.903570  18.400000  18.710714  15.959509   
       2013-01-08  18.900356  18.996071  18.616072  18.761070  16.002449   

                        Volume  
Ticker Date                     
AAPL   2013-01-02  560518000.0  
       2013-01-03  352965200.0  
       2013-01-04  594333600.0  
       2013-01-07  484156400.0  
       2013-01-08  458707200.0

In [17]:
# Also only pulling the ticker, date and adj. close columns for our tickers.

adj_close = all_data[["Adj Close"]].reset_index()
adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,16.725033
1,AAPL,2013-01-03,16.513929
2,AAPL,2013-01-04,16.053942
3,AAPL,2013-01-07,15.959509
4,AAPL,2013-01-08,16.002449


In [18]:
# Grabbing the ticker close from the end of last year
adj_close_start = adj_close[adj_close["Date"] == end_of_last_year]
adj_close_start.head()

,Ticker,Date,Adj Close
1258,AAPL,2017-12-29,39.901470
2563,JNJ,2017-12-29,117.916229
3868,MCD,2017-12-29,148.687073
5173,MTCH,2017-12-29,29.799627
6478,NFLX,2017-12-29,191.960007


In [19]:
# Grab the latest stock close price

adj_close_latest = adj_close[adj_close["Date"] == stocks_end]
adj_close_latest

,Ticker,Date,Adj Close


In [20]:
adj_close_latest.set_index("Ticker", inplace=True)
adj_close_latest.head()

,Date,Adj Close
Ticker,,


In [21]:
portfolio_df.set_index(["Ticker"], inplace=True)

portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29
MCD,2013-06-04,100,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29


In [22]:
# Merge the portfolio dataframe with the adj close dataframe; they are being joined by their indexes.

merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)
merged_portfolio.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close
Ticker,,,,,,,


In [23]:
# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.

merged_portfolio["ticker return"] = merged_portfolio["Adj Close"] / merged_portfolio["Unit Cost"] - 1

merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
Ticker,,,,,,,,


In [24]:
merged_portfolio.reset_index(inplace=True)

In [25]:
# Here we are merging the new dataframe with the sp500 adjusted closes since the sp start price based on
# each ticker's acquisition date and sp500 close date.

merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on="Acquisition Date", right_on="Date")
# .set_index('Ticker')

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date_x,Adj Close_x,ticker return,Date_y,Adj Close_y


In [26]:
# We will delete the additional date column which is created from this merge.
# We then rename columns to Latest Date and then reflect Ticker Adj Close and SP 500 Initial Close.

del merged_portfolio_sp["Date_y"]

merged_portfolio_sp.rename(columns={"Date_x": "Latest Date", "Adj Close_x": "Ticker Adj Close", "Adj Close_y": "SP 500 Initial Close"}, inplace=True)

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close


In [27]:
# This new column determines what SP 500 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp["Equiv SP Shares"] = merged_portfolio_sp["Cost Basis"] / merged_portfolio_sp["SP 500 Initial Close"]
merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares


In [28]:
# We are joining the developing dataframe with the sp500 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on="Latest Date", right_on="Date")

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,Date,Adj Close


In [29]:
# Once again need to delete the new Date column added as it's redundant to Latest Date.
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.

del merged_portfolio_sp_latest["Date"]

merged_portfolio_sp_latest.rename(columns={"Adj Close": "SP 500 Latest Close"}, inplace=True)

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close


In [30]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest["SP Return"] = merged_portfolio_sp_latest["SP 500 Latest Close"] / merged_portfolio_sp_latest["SP 500 Initial Close"] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest["Abs. Return Compare"] = merged_portfolio_sp_latest["ticker return"] - merged_portfolio_sp_latest["SP Return"]

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest["Ticker Share Value"] = merged_portfolio_sp_latest["Quantity"] * merged_portfolio_sp_latest["Ticker Adj Close"]

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest["SP 500 Value"] = merged_portfolio_sp_latest["Equiv SP Shares"] * merged_portfolio_sp_latest["SP 500 Latest Close"]

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest["Abs Value Compare"] = merged_portfolio_sp_latest["Ticker Share Value"] - merged_portfolio_sp_latest["SP 500 Value"]

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest["Stock Gain / (Loss)"] = merged_portfolio_sp_latest["Ticker Share Value"] - merged_portfolio_sp_latest["Cost Basis"]

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest["SP 500 Gain / (Loss)"] = merged_portfolio_sp_latest["SP 500 Value"] - merged_portfolio_sp_latest["Cost Basis"]

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)


In [31]:
# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
# Should not need to do the outer join;

merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on="Ticker")
# , how='outer'

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Date,Adj Close


In [32]:
# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.

del merged_portfolio_sp_latest_YTD["Date"]

merged_portfolio_sp_latest_YTD.rename(columns={"Adj Close": "Ticker Start Year Close"}, inplace=True)

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close


In [33]:
# Join the SP 500 start of year with current dataframe for SP 500 ytd comparisons to tickers.

merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start, left_on="Start of Year", right_on="Date")

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,Date,Adj Close


In [34]:
# Deleting another unneeded Date column.

del merged_portfolio_sp_latest_YTD_sp["Date"]

# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={"Adj Close": "SP Start Year Close"}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp["Share YTD"] = merged_portfolio_sp_latest_YTD_sp["Ticker Adj Close"] / merged_portfolio_sp_latest_YTD_sp["Ticker Start Year Close"] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp["SP 500 YTD"] = merged_portfolio_sp_latest_YTD_sp["SP 500 Latest Close"] / merged_portfolio_sp_latest_YTD_sp["SP Start Year Close"] - 1

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD


In [35]:
merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by="Ticker", ascending=True)
merged_portfolio_sp_latest_YTD_sp

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD


In [36]:
# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp["Cum Invst"] = merged_portfolio_sp_latest_YTD_sp["Cost Basis"].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp["Cum Ticker Returns"] = merged_portfolio_sp_latest_YTD_sp["Ticker Share Value"].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp["Cum SP Returns"] = merged_portfolio_sp_latest_YTD_sp["SP 500 Value"].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp["Cum Ticker ROI Mult"] = merged_portfolio_sp_latest_YTD_sp["Cum Ticker Returns"] / merged_portfolio_sp_latest_YTD_sp["Cum Invst"]

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult


## Assessing Where Positions are At versus Highest Close

In [37]:
# Referencing the adj_close dataframe from above

adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,16.725033
1,AAPL,2013-01-03,16.513929
2,AAPL,2013-01-04,16.053942
3,AAPL,2013-01-07,15.959509
4,AAPL,2013-01-08,16.002449


In [38]:
portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29
MCD,2013-06-04,100,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29


In [39]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace=True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on="Ticker")

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
0,AAPL,2013-01-02,16.725033,2013-02-07,125,65.4,8175.0,2017-12-29
1,AAPL,2013-01-03,16.513929,2013-02-07,125,65.4,8175.0,2017-12-29
2,AAPL,2013-01-04,16.053942,2013-02-07,125,65.4,8175.0,2017-12-29
3,AAPL,2013-01-07,15.959509,2013-02-07,125,65.4,8175.0,2017-12-29
4,AAPL,2013-01-08,16.002449,2013-02-07,125,65.4,8175.0,2017-12-29


In [40]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date["Quantity"]
del adj_close_acq_date["Unit Cost"]
del adj_close_acq_date["Cost Basis"]
del adj_close_acq_date["Start of Year"]

# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=["Ticker", "Acquisition Date", "Date"], ascending=[True, True, True], inplace=True)

In [41]:
# Anything less than 0 means that the stock close was prior to acquisition.
adj_close_acq_date["Date Delta"] = adj_close_acq_date["Date"] - adj_close_acq_date["Acquisition Date"]

adj_close_acq_date["Date Delta"] = adj_close_acq_date[["Date Delta"]].apply(pd.to_numeric)

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
0,AAPL,2013-01-02,16.725033,2013-02-07,-3110400000000000
1,AAPL,2013-01-03,16.513929,2013-02-07,-3024000000000000
2,AAPL,2013-01-04,16.053942,2013-02-07,-2937600000000000
3,AAPL,2013-01-07,15.959509,2013-02-07,-2678400000000000
4,AAPL,2013-01-08,16.002449,2013-02-07,-2592000000000000


In [42]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date["Date Delta"] >= 0]

adj_close_acq_date_modified.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
25,AAPL,2013-02-07,14.346462,2013-02-07,0
26,AAPL,2013-02-08,14.553590,2013-02-07,86400000000000
27,AAPL,2013-02-11,14.705256,2013-02-07,345600000000000
28,AAPL,2013-02-12,14.336651,2013-02-07,432000000000000
29,AAPL,2013-02-13,14.309386,2013-02-07,518400000000000


In [43]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=["Ticker", "Acquisition Date"], values="Adj Close", aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot

/tmp/ipykernel_1031/3678200190.py:3: FutureWarning:

The provided callable <function max at 0x7f2e941d5360> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.



,Ticker,Acquisition Date,Adj Close
0,AAPL,2013-02-07,42.370029
1,JNJ,2014-02-27,125.022278
2,MCD,2013-06-04,154.077499
3,MTCH,2015-12-14,42.220104
4,NFLX,2016-01-14,325.220001
5,WMT,2013-08-14,32.600822


In [44]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close, on=["Ticker", "Adj Close"])

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,AAPL,2013-02-07,42.370029,2018-02-26
1,JNJ,2014-02-27,125.022278,2018-01-22
2,MCD,2013-06-04,154.077499,2018-01-26
3,MTCH,2015-12-14,42.220104,2018-03-08
4,NFLX,2016-01-14,325.220001,2018-03-06


In [45]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged, on=["Ticker", "Acquisition Date"])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={"Adj Close": "Closing High Adj Close", "Date": "Closing High Adj Close Date"}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high["Pct off High"] = (
    merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker Adj Close"] / merged_portfolio_sp_latest_YTD_sp_closing_high["Closing High Adj Close"] - 1
)

merged_portfolio_sp_latest_YTD_sp_closing_high

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High


In [46]:
# Not needed for this blog post -- this is if you have multiple positions for the same ticker with different acquisition dates.
# merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'] = merged_portfolio_sp_latest_YTD_sp_closing_high.index

# merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker #'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'].map(str) + ' ' + merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'].map(str)

# merged_portfolio_sp_latest_YTD_sp_closing_high.head()

## YTD and Trailing Stop Charts

In [47]:
# Ploty is an outstanding resource for interactive charts.

trace1 = go.Bar(x=merged_portfolio_sp_latest_YTD_sp["Ticker"][0:10], y=merged_portfolio_sp_latest_YTD_sp["Share YTD"][0:10], name="Ticker YTD")

trace2 = go.Scatter(x=merged_portfolio_sp_latest_YTD_sp["Ticker"][0:10], y=merged_portfolio_sp_latest_YTD_sp["SP 500 YTD"][0:10], name="SP500 YTD")

data = [trace1, trace2]

layout = go.Layout(title="YTD Return vs S&P 500 YTD", barmode="group", yaxis=dict(title="Returns", tickformat=".2%"), xaxis=dict(title="Ticker"), legend=dict(x=0.8, y=1))

fig = go.Figure(data=data, layout=layout)
iplot(fig)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Current Share Price versus Closing High Since Purchased

trace1 = go.Bar(x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"][0:10], y=merged_portfolio_sp_latest_YTD_sp_closing_high["Pct off High"][0:10], name="Pct off High")

data = [trace1]

layout = go.Layout(
    title="Adj Close % off of High", barmode="group", yaxis=dict(title="% Below Adj Close High", tickformat=".2%"), xaxis=dict(title="Ticker"), legend=dict(x=0.8, y=1)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Total Return Comparison Charts

In [ ]:
trace1 = go.Bar(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"][0:10], y=merged_portfolio_sp_latest_YTD_sp_closing_high["ticker return"][0:10], name="Ticker Total Return"
)

trace2 = go.Scatter(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"][0:10], y=merged_portfolio_sp_latest_YTD_sp_closing_high["SP Return"][0:10], name="SP500 Total Return"
)

data = [trace1, trace2]

layout = go.Layout(
    title="Total Return vs S&P 500", barmode="group", yaxis=dict(title="Returns", tickformat=".2%"), xaxis=dict(title="Ticker", tickformat=".2%"), legend=dict(x=0.8, y=1)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Cumulative Returns Over Time

In [ ]:
trace1 = go.Bar(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"][0:10], y=merged_portfolio_sp_latest_YTD_sp_closing_high["Stock Gain / (Loss)"][0:10], name="Ticker Total Return ($)"
)

trace2 = go.Bar(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"][0:10], y=merged_portfolio_sp_latest_YTD_sp_closing_high["SP 500 Gain / (Loss)"][0:10], name="SP 500 Total Return ($)"
)

trace3 = go.Scatter(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"][0:10],
    y=merged_portfolio_sp_latest_YTD_sp_closing_high["ticker return"][0:10],
    name="Ticker Total Return %",
    yaxis="y2",
)

data = [trace1, trace2, trace3]

layout = go.Layout(
    title="Gain / (Loss) Total Return vs S&P 500",
    barmode="group",
    yaxis=dict(title="Gain / (Loss) ($)"),
    yaxis2=dict(title="Ticker Return", overlaying="y", side="right", tickformat=".2%"),
    xaxis=dict(title="Ticker"),
    legend=dict(x=0.75, y=1),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
trace1 = go.Bar(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"],
    y=merged_portfolio_sp_latest_YTD_sp_closing_high["Cum Invst"],
    # mode = 'lines+markers',
    name="Cum Invst",
)

trace2 = go.Bar(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"],
    y=merged_portfolio_sp_latest_YTD_sp_closing_high["Cum SP Returns"],
    # mode = 'lines+markers',
    name="Cum SP500 Returns",
)

trace3 = go.Bar(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"],
    y=merged_portfolio_sp_latest_YTD_sp_closing_high["Cum Ticker Returns"],
    # mode = 'lines+markers',
    name="Cum Ticker Returns",
)

trace4 = go.Scatter(
    x=merged_portfolio_sp_latest_YTD_sp_closing_high["Ticker"],
    y=merged_portfolio_sp_latest_YTD_sp_closing_high["Cum Ticker ROI Mult"],
    # mode = 'lines+markers',
    name="Cum ROI Mult",
    yaxis="y2",
)


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(
    title="Total Cumulative Investments Over Time",
    barmode="group",
    yaxis=dict(title="Returns"),
    xaxis=dict(title="Ticker"),
    legend=dict(x=0.4, y=1),
    yaxis2=dict(title="Cum ROI Mult", overlaying="y", side="right"),
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)